# Electron degradation

### Exploring the different contributions to the energy loss rate for accelerated electrons.

We start with a bunch of constants and utility functions:

In [97]:
'''
This code outputs the energy loss rates following Barnes&Kasen2016
(ApJ 829:110, 2016)
Section 3: Thermalisation physics.
Processes by which energetic decay products thermalize in the KN ejecta.
Focus on beta-particles.
(Taken from ./energyLossrates.py)
'''

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.interpolate import interp1d
import sys
import textwrap

# Constants
lambda_ee_ = 10  # Coulomb logarithm for electron-electron scattering
m_e_ = 9.10938356e-28  # electron mass (g)
m_p_ = 1.6726219e-24 # proton mass (g)
r_e_ = 2.8179403227e-13  # classical electron radius (cm)
m_u_ = 1.660539e-24  # nuclear mass unit (g)
c_ = 2.99792458e10  # light speed (cm/s)
kB_ = 8.617333262145e-11  # Boltzmann constant (MeV/K)
Msun_ = 2.0e33 # Solar mass (g)
r_0_ = 1.8e-13 # nuclear radius coef for deformed post-scission nuclei (cm)
alpha_ = 1./137. # fine structure constant 

delta_ = 1  # power-law profile for inner ejecta (Barnes&Kasen2016)
n_ = 1  # power-law profile for outer ejecta (Barnes&Kasen2016)


def to_erg(E):
    return(E*1.60218e-6)


def to_MeV(E):
    return(E/1.60218e-6)

#### Plasma losses
Coulomb interactions with free thermal electrons. In the limit that the $\beta$-particle energy far exceeds that of thermal electrons, **Huba2013** gives:

In [98]:
def plasma_losses(E_beta, n_e, T):
    ''' Returns beta_particle losses from interaction with free thermal
    electrons (MeV/s).
    args:
    E_beta = beta-particle kinetic energy (MeV)
    n_e = free electron number density (cm-3)
    T = ejecta temperature (MeV)
    '''

    return(7.7e-15 
           * E_beta**(-.5) 
           * n_e * lambda_ee_ 
           * (1. - (3.9 / 7.7) * T / E_beta))

This assumes that the dominating interactions are electron/electron (ignoring electron/ion interactions). This depends on the density of free thermal electrons, and thus the composition of the medium. It is not clear in Barnes&Kasen2016 whether they assume that all free thermal electrons only come from atoms heavier than hydrogen.

**The composition does not evolve significantly over the course of 1-10 days.**

- Inner ejecta: single component mean composition Z=60 A=144
- Outer ejecta: 2-component: Hydrogen Z=1 A=1, heavy elems Z=55 A=133\
But that hydrogen does not contribute to plasma because it is only singly ionised. Decray of neutrons into protons. A says it should be a small mass component.

Axelrod has discussion on this.

#### IE losses
When $\beta$-particles are exciting or inizing bound atomic electrons.

In [99]:
def calc_n_eb(rho, z_over_a):
    ''' Returns the bound electron number density (cm-3) for a given
    average Z/A.
    args:
    rho = mass density (g/cm-3)
    z_over_a = composition (0.4 inner / 0.55 outer ejecta)
    '''
    return(rho / m_u_ * z_over_a)


def IE_losses(E_beta, Ibar, v_beta, n_eb):
    ''' Returns beta_particle losses due to ionization and excitation
    of atomic electrons (MeV/s).
    args:
    E_beta = beta-particle kinetic energy (MeV)
    I_bar = average ionization and excitation potential (MeV) (eq. 9)
    v_beta = beta-particle speed (cm/s)
    n_eb = bound electron number density (cm-3) 
    '''

    tau = to_erg(E_beta) / (m_e_ * c_**2) 
    g = 1 + tau**2 / 8 - (2*tau+1) * np.log(2)
    Edot_IE = 2*np.pi * r_e_**2 * m_e_ * c_**3 * n_eb / (v_beta/c_)
    Edot_IE *= (2 * np.log(E_beta/Ibar) 
                + np.log(1 + tau/2) 
                + (1 - (v_beta/c_)**2) * g)

    return(to_MeV(Edot_IE))

#### Synchrotron losses
This is strongly dependent on the magnetic field. This is negligible in the kilonova for the expected magnetic field values of 3.6e-6G at 1 day (extracted from eq. 12, Barnes&Kasen2016)

In [100]:
def synch_losses(v_beta, B):
    ''' Returns synchrotron losses from Barnes&Kasen2016 eq. 11. (MeV/s)
    args:
    v_beta = beta particle velocity (cm/s)
    B = Magnetic field strength (G)
    '''
    gamma = np.sqrt(1./(1-(v_beta/c_)**2))
    loss = (4./9.) * r_e_**2 * c_ * gamma**2 * (v_beta / c_)**2 * B**2
    return(to_MeV(loss))

#### Bremsstrahlung
This is tricky! Strongly depends on the composition. Assumptions have to be made on the average atomic number of the ejecta, which will also vary depending on whether you are in the inner or outer ejecta. Cross-sections have been tabulated by **Seltzer&Berger1986**.

In [139]:
def phirad(E_beta, Z):
    ''' Returns the scaled total integrated radiative energy-loss 
    cross-section from Bremsstrahlung (Seltzer&Berger1986)
    Notation: phirad
    args:
    E_beta = electron kinetic energy (MeV)
    Z = Atomic number
        (characteristic inner ejecta values Z = 60; A = 144)
        (characteristic outer ejecta values, two components
        Z = 1; A = 1 (hydrogen) and Z = 55; A = 133 (heavier elems))
    '''

    filename = "phirad_Z%d.csv" % Z
    try:
        data = pd.read_csv(filename, delimiter=" ")
    except FileNotFoundError:
        print(textwrap.dedent("""
              %s File not found. 
              You are probably asking for a value of Z that is not tabulated here. 
              See Seltzer&Berger1986 for tabulations for all values.
              """ % filename))
        return(-1)
    
    T1_arr = data.T1.to_numpy() # in MeV, so simply E_beta
    phirad_arr = data.phirad.to_numpy()
    
    def log_interp1d(xx, yy, kind='linear'):
        logx = np.log10(xx)
        logy = np.log10(yy)
        lin_interp = interp1d(logx, logy, kind=kind)
        log_interp = lambda zz: np.power(10.0, lin_interp(np.log10(zz)))
        return log_interp
    
    phirad_interp = log_interp1d(T1_arr, phirad_arr)
    
    return(phirad_interp(E_beta))


def Brem_losses(n_i, v_beta, E_beta, Z):
    ''' Returns losses due to Bremsstrahlung from 
    Barnes&Kasen2016 eq.13 (MeV)
    n_i = number density of the scattering species (cm-3)
    v_beta = beta particle velocity (cm/s)
    E_beta = Beta particle kinetic energy (MeV)
    Z = Atomic number 
        (characteristic inner ejecta values Z = 60; A = 144)
        (characteristic outer ejecta values, two components
        Z = 1; A = 1 (hydrogen) and Z = 55; A = 133 (heavier elems))
    '''
    E_dot = (n_i * v_beta * (to_erg(E_beta) + m_e_*c_**2)
             * Z**2 * r_0_**2 * alpha_ * phirad(E_beta, Z))
    return(to_MeV(E_dot))

### Plotting loss rates for typical values

Ejecta mass: $M_\mathrm{ej} = 10^{-3} M_\odot$ \
Ejecta velocity: $v_\mathrm{ej} = 0.2c$

We look at $t=1$day.

Ejecta density = $M_\mathrm{ej} / \frac{4}{3} \pi (v_\mathrm{ej}t)^3 = 1.7 \times 10^{14}$ g.cm$^{-2}$


In [140]:
M_ej = 5.e-3 * Msun_
v_ej = 0.2 * c_
t = 1 * 86400.
rho_ej = M_ej / ((4./3.) * np.pi * (v_ej * t)**3)
print("%e" %rho_ej)

1.717189e-14


For a given mass density and mass number A, we can compute the number density of scattering species for Bremsstrahlung:

In [141]:
A = 144 # inner ejecta
n_i = rho_ej / (A*m_p_)

We can now compute the contributions from plasma, IE and Bremsstrahlung losses. We start by setting up the KN parameters

In [142]:
E_beta = np.logspace(-2, 1, 100) # particle kinetic energy (MeV)
T = 1e-6 # MeV
Z = 60  # for inner ejecta
A = 144  #for inner ejecta

n_e = rho_ej / (A * m_p_) * (1./Z)  # number density of bound electrons
    # elements heavier than H are singly ionized (low temperatures)
    # Composition from Barnes+2016 fig.1 (took A_heavy=1.e-3)
    # (this can be refined)
    
Ibar = np.exp(6.4) * 1.e-6 # (MeV)
gamma_beta = 1 + to_erg(E_beta) / (m_e_ * c_**2)
v_beta = np.sqrt(1 - 1./gamma_beta**2) * c_
rho = rho_ej
B = 3.7e-6

This is the plotting function

In [143]:
def plot_rates(T, rho, B, Z, A):
    
    n_eb = calc_n_eb(rho, Z/A)
    n_e = rho / m_p_ *1.e-3
    
    plasma = plasma_losses(E_beta, n_e, T) / rho
    IE = IE_losses(E_beta, Ibar, v_beta, n_eb) / rho
    synch = synch_losses(v_beta, B) / rho
    Brem = Brem_losses(n_i, v_beta, E_beta, Z) / rho
    tot = plasma + IE + synch + Brem
    
    plt.figure()
    plt.plot(E_beta, plasma, label='plasma')
    plt.plot(E_beta, IE, label = 'IE')
    plt.plot(E_beta, synch, label = 'synchrotron')
    plt.plot(E_beta, Brem, label='Bremsstrahlung')
    plt.plot(E_beta, tot, label='total')

    plt.xscale('log')
    plt.yscale('log')
    plt.legend()
    plt.show()

In [145]:
%matplotlib widget
plot_rates(T, rho, B, Z, A)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Parameter exploration

We ll take a look at the losses as a function of $T$, $\rho$ and $B$.

The questions are:
- Can synchrotron dominate at some point in KN evolution?
- Can something else than Synchrotron dominate at some point in GRB afterglows?

In [148]:
%matplotlib inline
from ipywidgets import *

Tpe = widgets.Select(description='Ejecta type', 
                     options=['Hydrogen', 'Inner ejecta'], 
                     value='Inner ejecta')
T = widgets.FloatLogSlider(description='T', min=-10, max=0, value=1.e-6, step=1)
rho = widgets.FloatLogSlider(description='rho', min=-18, max=-10, value=1.e-14, step=1)
B = widgets.FloatLogSlider(description='B', min=-10, max=5, value=1.e-6, step=1)
Z = widgets.Dropdown(description='Z', options=[1,60], value=60)
A = widgets.Dropdown(description='A', options=[1,133,144], value=144)
params_ui = widgets.VBox([Tpe, T, rho, B, Z, A])

def update_Z_A(*args):
    if Tpe.value == 'Hydrogen':
        Z.value = 1
        A.value = 1
    if Tpe.value == 'Inner ejecta':
        Z.value = 60
        A.value = 144
    if Tpe.value == 'Outer':
        Z.value = 55
        A.value = 133
Tpe.observe(update_Z_A, 'value')

def update(Tpe, T, rho, B, Z, A):
    plot_rates(T, rho, B, Z, A)
        
out = widgets.interactive_output(update, {'Tpe':Tpe, 'T': T, 'rho': rho, 'B': B, 'Z':Z, 'A':A})

ui = widgets.HBox([out, params_ui])
display(ui)


Now we need to evolve this following Meztger review for temperature and density.

A: Goes into homologuous expansion: $\rho \propto t^{-3}$. Quentin's work suggests that T might be increasing. In some models. T might go up to $10^{5}K$, but probably not above. 

hello